In [2]:
# Imports and installs
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from zipfile import ZipFile # library for handling ZIP files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge geocoder --yes
import geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from folium import plugins

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### Download Dutch area geolocation data based on postal code

Dutch postal codes have 6 positions: four digits followed by two letters. The four digits are the best Dutch proxy of neighbourhoods.

Geolocation data sets for the four digits postal code (4PP) can amongst others be found at https://git.tuxm.nl/tuxmachine/postcodes/src/4329c858db24b79523fd3fbbaf2df138ccaf16cd

Credit: https://git.tuxm.nl/tuxmachine/postcodes/src/master/README.md

License: https://git.tuxm.nl/tuxmachine/postcodes/src/master/LICENSE

In [3]:
#4PP Polygons in JSON dataset for Netherlands areas in polygon geolocation
!wget -q -O '4pp.polygons.json' https://git.tuxm.nl/tuxmachine/postcodes/raw/4329c858db24b79523fd3fbbaf2df138ccaf16cd/4pp.polygons.json
print('Data downloaded!')

Data downloaded!


In [4]:
#4PP LatLong in CSV dataset for Netherlands areas in LatLong geolocation
!wget -q -O '4pp.csv' https://git.tuxm.nl/tuxmachine/postcodes/src/master/4pp.csv
print('Data downloaded!')

Data downloaded!


In [5]:
neighlatlong = pd.read_csv('https://git.tuxm.nl/tuxmachine/postcodes/raw/master/4pp.csv')
neighlatlong.head()

,id,postcode,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
0,1,1000,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.336243,4.869444,Postbus
1,2,1001,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364240,4.883358,Postbus
2,3,1002,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364240,4.883358,Onbekend
3,4,1003,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364240,4.883358,Onbekend
4,5,1005,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.364240,4.883358,Postbus


In [6]:
neighlatlong.shape

(4699, 10)

Drop all areas with category ('Soort') Postbus (=P.O. Box) or Onbekend (=Unkown)

In [7]:
#Drop all rows with Soort is Onbekend
neighlatlong.drop(neighlatlong[neighlatlong.soort == 'Onbekend'].index, inplace=True)
#Drop all rows with Soort is Postbus
neighlatlong.drop(neighlatlong[neighlatlong.soort == 'Postbus'].index, inplace=True)
neighlatlong.head()

,id,postcode,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
9,10,1011,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372976,4.903957,Adres
10,11,1012,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.373386,4.894064,Adres
11,12,1013,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.396789,4.876607,Adres
12,13,1014,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.392305,4.855884,Adres
13,14,1015,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.379093,4.885109,Adres


In [8]:
neighlatlong.shape

(4066, 10)

### CBS names of areas and neighbourhoods based on postal code

The Dutch Central Bureau of Statistics (CBS) has an open dataset on areas and neighbourhoods based on postal code.

See: https://www.cbs.nl/nl-nl/maatwerk/2018/36/buurt-wijk-en-gemeente-2018-voor-postcode-huisnummer

In [9]:
#Download and extract ZIP file of CBS area and neighbourhood data
!wget -q -O '2018-cbs-pc6huisnr20180801_buurt-vs2.zip' https://www.cbs.nl/-/media/_excel/2018/36/2018-cbs-pc6huisnr20180801_buurt%20-vs2.zip
with ZipFile('2018-cbs-pc6huisnr20180801_buurt-vs2.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()
print('Data downloaded!')

Data downloaded!


In [10]:
#Create dataframe on lowlevel postal code data
pc = pd.read_csv('pc6hnr20180801_gwb-vs2.csv', sep=';', encoding='latin_1')
pc.rename(columns = {'Buurt2018': 'buurtcode', 'Wijk2018': 'wijkcode', 'Gemeente2018': 'gemeentecode'}, inplace = True)
pc.head()

,PC6,Huisnummer,buurtcode,wijkcode,gemeentecode
0,1011AB,105,3630400,36304,363
1,1011AB,106,3630400,36304,363
2,1011AB,107,3630400,36304,363
3,1011AB,110,3630400,36304,363
4,1011AB,112,3630400,36304,363


In [11]:
#Create dataframe with neighbourhood codes and names
wijk = pd.read_csv('wijknaam2018.csv', sep=';', encoding='latin_1')
wijk.rename(columns = {'GWBcode8': 'wijkcode', 'GWBlabel': 'wijk'}, inplace = True)
wijk.head()

,wijkcode,wijk
0,300,Wijk 00
1,500,Wijk 00
2,900,Wijk 00 West
3,901,Wijk 01 Oost
4,1000,Wijk 00 Stad


In [12]:
#Create dataframe with area codes and names
buurt = pd.read_csv('buurtnaam2018.csv', sep=';', encoding='latin_1')
buurt.rename(columns = {'GWBcode8': 'buurtcode', 'GWBlabel': 'buurt'}, inplace = True)
buurt.head()

,buurtcode,buurt
0,30000,Appingedam-Centrum
1,30001,Appingedam-West
2,30002,Appingedam-Oost
3,30007,Verspreide huizen Damsterdiep en Eemskanaal
4,30008,Verspreide huizen ten zuiden van Eemskanaal


In [13]:
#Merge postal code dataframe with neighbourhood and area names
pcmerge = pd.merge(pc, wijk, on='wijkcode')
pcmerge = pd.merge(pcmerge, buurt, on='buurtcode')
pcmerge.head()

,PC6,Huisnummer,buurtcode,wijkcode,gemeentecode,wijk,buurt
0,1011AB,105,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland
1,1011AB,106,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland
2,1011AB,107,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland
3,1011AB,110,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland
4,1011AB,112,3630400,36304,363,Nieuwmarkt/Lastage,Oosterdokseiland


In [14]:
#Drop columns we will not need: housenumber (too low level), Municipality (will be added by geocode data by name instead of code) and the codes
pcmerge.drop(['Huisnummer', 'buurtcode', 'wijkcode', 'gemeentecode'], axis=1, inplace = True)
pcmerge.head()

,PC6,wijk,buurt
0,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland
1,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland
2,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland
3,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland
4,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland


In [15]:
#Create column with first four digits of postal code as all six is too low level
pcmerge['postcode'] = pcmerge['PC6'].str[:4]
pcmerge.head()

,PC6,wijk,buurt,postcode
0,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011
1,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011
2,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011
3,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011
4,1011AB,Nieuwmarkt/Lastage,Oosterdokseiland,1011


In [16]:
#Drop PC6 and group on postcode
postcode = pcmerge.groupby(['postcode'], as_index=False).first()
postcode.drop(['PC6'], axis=1, inplace = True)
postcode.head()

,postcode,wijk,buurt
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.
2,1013,Haarlemmerbuurt,Westerdokseiland
3,1014,Westelijk Havengebied,Alfa-driehoek
4,1015,Grachtengordel-West,Langestraat e.o.


In [17]:
#Cast column postcode to INT instead of OBJECT as geolocation dataset has INT
postcode.postcode = postcode.postcode.astype('int64')

In [18]:
#Now merge with long/lat
postcode = pd.merge(postcode, neighlatlong, on='postcode')
postcode.head()

,postcode,wijk,buurt,id,woonplaats,alternatieve_schrijfwijzen,gemeente,provincie,netnummer,latitude,longitude,soort
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland,10,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.372976,4.903957,Adres
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,11,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.373386,4.894064,Adres
2,1013,Haarlemmerbuurt,Westerdokseiland,12,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.396789,4.876607,Adres
3,1014,Westelijk Havengebied,Alfa-driehoek,13,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.392305,4.855884,Adres
4,1015,Grachtengordel-West,Langestraat e.o.,14,Amsterdam,NaN,Amsterdam,Noord-Holland,20,52.379093,4.885109,Adres


In [19]:
#Drop unneeded columns
postcode.drop(['id', 'alternatieve_schrijfwijzen', 'soort'], axis=1, inplace = True)
postcode.head()

,postcode,wijk,buurt,woonplaats,gemeente,provincie,netnummer,latitude,longitude
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland,Amsterdam,Amsterdam,Noord-Holland,20,52.372976,4.903957
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,Amsterdam,Amsterdam,Noord-Holland,20,52.373386,4.894064
2,1013,Haarlemmerbuurt,Westerdokseiland,Amsterdam,Amsterdam,Noord-Holland,20,52.396789,4.876607
3,1014,Westelijk Havengebied,Alfa-driehoek,Amsterdam,Amsterdam,Noord-Holland,20,52.392305,4.855884
4,1015,Grachtengordel-West,Langestraat e.o.,Amsterdam,Amsterdam,Noord-Holland,20,52.379093,4.885109


In [20]:
postcode.shape

(4026, 9)

### Show neighbourhoods on map for the cities of Amsterdam, Rotterdam, The Hague and Utrecht

In [28]:
# Obtain latitude and longitude of the Netherlands
g = geocoder.arcgis('Utrecht, Netherlands')
Utrecht_coords = g.latlng
print("Utrecht", Utrecht_coords)
g = geocoder.arcgis('Amsterdam, Netherlands')
amsterdam_coords = g.latlng
print("Amsterdam", amsterdam_coords)
g = geocoder.arcgis('Rotterdam, Netherlands')
rotterdam_coords = g.latlng
print("Rotterdam", rotterdam_coords)
g = geocoder.arcgis('Den Haag, Netherlands')
denhaag_coords = g.latlng
print("The Hague", denhaag_coords)

Utrecht [52.08965000000006, 5.114350000000059]
Amsterdam [52.36993000000007, 4.907880000000034]
Rotterdam [51.92282000000006, 4.478480000000047]
The Hague [52.08409000000006, 4.317320000000052]


In [22]:
#Extract only woonplaats is Utrecht
utrecht = postcode[postcode.woonplaats.str.contains('utrecht',case=False)]
utrecht.head()

,postcode,wijk,buurt,woonplaats,gemeente,provincie,netnummer,latitude,longitude
1075,3511,Wijk 06 Binnenstad,"Lange Elisabethstraat, Mariaplaats en omgeving",Utrecht,Utrecht,Utrecht,30,52.089633,5.117682
1076,3512,Wijk 06 Binnenstad,Breedstraat en Plompetorengracht en omgeving,Utrecht,Utrecht,Utrecht,30,52.089348,5.124238
1077,3513,Wijk 06 Binnenstad,Hoog-Catharijne NS en Jaarbeurs,Utrecht,Utrecht,Utrecht,30,52.097969,5.109309
1078,3514,Wijk 04 Noordoost,Lauwerecht,Utrecht,Utrecht,Utrecht,30,52.100037,5.122910
1079,3515,Wijk 04 Noordoost,Lauwerecht,Utrecht,Utrecht,Utrecht,30,52.102959,5.116839


In [24]:
#Extract only woonplaats is Amsterdam
amsterdam = postcode[postcode.woonplaats.str.contains('amsterdam',case=False)]
amsterdam.head()

,postcode,wijk,buurt,woonplaats,gemeente,provincie,netnummer,latitude,longitude
0,1011,Nieuwmarkt/Lastage,Oosterdokseiland,Amsterdam,Amsterdam,Noord-Holland,20,52.372976,4.903957
1,1012,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,Amsterdam,Amsterdam,Noord-Holland,20,52.373386,4.894064
2,1013,Haarlemmerbuurt,Westerdokseiland,Amsterdam,Amsterdam,Noord-Holland,20,52.396789,4.876607
3,1014,Westelijk Havengebied,Alfa-driehoek,Amsterdam,Amsterdam,Noord-Holland,20,52.392305,4.855884
4,1015,Grachtengordel-West,Langestraat e.o.,Amsterdam,Amsterdam,Noord-Holland,20,52.379093,4.885109


In [25]:
#Extract only woonplaats is The Hague
denhaag = postcode[postcode.woonplaats.str.contains('den haag',case=False)]
denhaag.head()

,postcode,wijk,buurt,woonplaats,gemeente,provincie,netnummer,latitude,longitude
627,2491,Wijk 44 Leidschenveen,De Velden,Den Haag,Den Haag,Zuid-Holland,70,52.069918,4.387614
628,2492,Wijk 44 Leidschenveen,De Velden,Den Haag,Den Haag,Zuid-Holland,70,52.063672,4.412687
629,2493,Wijk 44 Leidschenveen,De Lanen,Den Haag,Den Haag,Zuid-Holland,70,52.057377,4.403541
630,2495,Wijk 41 Hoornwijk,Vlietzoom-West,Den Haag,Den Haag,Zuid-Holland,70,52.056302,4.361045
631,2496,Wijk 41 Hoornwijk,De Reef,Den Haag,Den Haag,Zuid-Holland,70,52.049285,4.371331


In [26]:
#Extract only woonplaats is Rotterdam
rotterdam = postcode[postcode.woonplaats.str.contains('rotterdam',case=False)]
rotterdam.head()

,postcode,wijk,buurt,woonplaats,gemeente,provincie,netnummer,latitude,longitude
825,3011,Rotterdam Centrum,Stadsdriehoek,Rotterdam,Rotterdam,Zuid-Holland,10,51.917658,4.486852
826,3012,Rotterdam Centrum,Stadsdriehoek,Rotterdam,Rotterdam,Zuid-Holland,10,51.919405,4.475769
827,3013,Rotterdam Centrum,Cs Kwartier,Rotterdam,Rotterdam,Zuid-Holland,10,51.924289,4.469251
828,3014,Rotterdam Centrum,Cool,Rotterdam,Rotterdam,Zuid-Holland,10,51.919772,4.466089
829,3015,Rotterdam Centrum,Oude Westen,Rotterdam,Rotterdam,Zuid-Holland,10,51.911750,4.468044


In [29]:
#Create map of Utrecht with neighbourhoods marked
map_utrecht = folium.Map(location=[Utrecht_coords[0], Utrecht_coords[1]], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(utrecht['latitude'], utrecht['longitude'], utrecht['wijk'], utrecht['buurt']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_utrecht)  
    
map_utrecht

In [30]:
#Create map of Amsterdam with neighbourhoods marked
map_amsterdam = folium.Map(location=[amsterdam_coords[0], amsterdam_coords[1]], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(amsterdam['latitude'], amsterdam['longitude'], amsterdam['wijk'], amsterdam['buurt']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_amsterdam)  
    
map_amsterdam

In [31]:
#Create map of Den Haag with neighbourhoods marked
map_denhaag = folium.Map(location=[denhaag_coords[0], denhaag_coords[1]], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(denhaag['latitude'], denhaag['longitude'], denhaag['wijk'], denhaag['buurt']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_denhaag)  
    
map_denhaag

In [32]:
#Create map of Rotterdam with neighbourhoods marked
map_rotterdam = folium.Map(location=[rotterdam_coords[0], rotterdam_coords[1]], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(rotterdam['latitude'], rotterdam['longitude'], rotterdam['wijk'], rotterdam['buurt']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_rotterdam)  
    
map_rotterdam